## 7. 패딩(Padding)
- 데이터에 특정 값을 채워서 데이터의 크기(shape)를 조정하는 것
- 길이가 전부 동일한 문서들에 대해서는 하나의 행렬로 보고, 한꺼번에 묶어서 처리가 가능
* -> 병렬 연산을 위해서 여러 문장의 길이를 임의로 동일하게 맞춰주는 기법이 필요 

### 1. Numpy로 패딩

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [8]:
sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

정수 인코딩

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences) # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

텍스트 시퀀스의 모든 단어들을 각 정수로 맵핑 후 출력

In [10]:
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


↑모든 단어가 고유한 정수로 변환됨

모두 동일한 길이로 맞춰주기 위해서 이 중에서 가장 길이가 긴 문장의 길이를 계산

In [12]:
max_len = max(len(item) for item in encoded)
print(max_len)

7


* 가장 길이가 긴 문장의 길이는 7
* 모든 문장의 길이를 7로 맞춰 가상의 단어 'PAD'를 사용
* 'PAD'라는 단어가 있다고 가정하고, 이 단어는 0번 단어라고 정의
* 길이가 7보다 짧은 문장에는 숫자 0을 채워서 전부 길이 7로 맞춤

In [16]:
for item in encoded: # 각 문장에 대해서
    while len(item) < max_len:   # max_len보다 작으면
        item.append(0)

padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

* 길이가 7보다 짧은 문장에는 전부 숫자 0이 뒤로 붙음
*  숫자 0을 사용하고 있다면 제로 패딩(zero padding)이라함
* 이후, 하나의 행렬로 보고, 병렬 처리가 가능해짐
#### 이와 같이 데이터에 특정 값을 채워서 데이터의 크기(shape)를 조정하는 것을 패딩(padding)이라고 한다. 

### 2. 케라스 전처리 도구로 패딩하기
케라스에서 패딩을 위한 도구 pad_sequences()를 제공

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded 값이 위에서 이미 패딩 후의 결과로 저장되었으므로, 패딩 이전의 값으로 다시 되돌리는 과정을 추가

In [21]:
encoded = tokenizer.texts_to_sequences(sentences)
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [22]:
# 케라스의 pad_sequences를 사용하여 패딩
padded = pad_sequences(encoded)
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]])

Numpy로 패딩을 진행하였을 때와는 패딩 결과가 다른데 그 이유는 pad_sequences는 기본적으로 문서의 뒤에 0을 채우는 것이 아니라 앞에 0으로 채우기 때문

* 뒤에 0을 채우고 싶다면 인자로 padding='post'

In [24]:
padded = pad_sequences(encoded, padding = 'post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

Numpy를 이용하여 패딩을 했을 때와 결과가 동일한지 두 결과를 비교

In [26]:
(padded == padded_np).all()

True

꼭 가장 긴 문서의 길이를 기준으로 해야하는 것은 아니며, 길이에 제한을 두고 패딩할 수 있다. 
* 문서의 길이가 큰 경우에는 max_len의 인자로 정수를 주어, 해당 정수로 모든 문서의 길이를 동일하게 한다.

In [28]:
padded = pad_sequences(encoded, padding = 'post', maxlen = 5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]])

길이가 5보다 짧은 문서들은 0으로 패딩되고, 기존에 5보다 길었다면 데이터가 손실됨
숫자 0으로 패딩하는 것은 널리 퍼진 관례이긴 하지만, 반드시 지켜야하는 규칙은 아님
#### 숫자 0이 아니라 다른 숫자를 패딩을 위한 숫자로 사용하는 예시
* 현재 사용된 정수들과 겹치지 않도록, 단어 집합의 크기에 +1

In [30]:
last_value = len(tokenizer.word_index) + 1 # 단어 집합의 크기보다 1 큰 숫자를 사용
print(last_value)

14


* 현재 단어가 총 13개
* 단어 집합의 크기 +1 = 마지막 숫자인 13보다 1이 큰 14 
#### pad_sequences의 인자로 value를 사용하면 0이 아닌 다른 숫자로 패딩 가능

In [32]:
padded = pad_sequences(encoded, padding = 'post', value = last_value)
padded

array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]])